### 1.Import library

In [1]:
import json
from langchain_huggingface import HuggingFaceEmbeddings
import os
from langchain_experimental.text_splitter import SemanticChunker
from langchain.docstore.document import Document
from langchain_qdrant import QdrantVectorStore, RetrievalMode
from fastembed import  SparseTextEmbedding
from dotenv import load_dotenv

### 2.Store

> Load json

In [2]:
# Đọc dữ liệu từ tệp JSON
with open('../data/data_origin/output_data.json', 'r', encoding='utf-8') as json_file:
    documents = json.load(json_file)

In [3]:
# Kiểm tra nội dung JSON đã tạo
print(json.dumps(documents[:2], ensure_ascii=False, indent=2))  # In ra 5 tài liệu đầu tiên

[
  {
    "id": 1,
    "keyword": "Tư tưởng Hồ Chí Minh",
    "keyword_sub": "không có nguyên lý",
    "word_type": "động từ tiêu cực",
    "link": "https://www.vietnamplus.vn/gia-tri-ly-luan-va-thuc-tien-cua-tu-tuong-ho-chi-minh-post235139.vnp",
    "content": "Giá trị lý luận và thực tiễn của tư tưởng Hồ Chí Minh\nTTXVN giới thiệu bài viết của Phó giáo sư-tiến sỹ Nguyễn Hoàng Giáp với nhan đề “Giá trị lý luận và thực tiễn của tư tưởng Hồ Chí Minh-Vấn đề không thể phủ nhận.”\nGần đây, trên một số trang thông tin có đăng tải bài viết với nội dung luận bàn về “Những ảo tưởng của Đảng Cộng sản Việt Nam về tư tưởng Hồ Chí Minh”; hoặc: “Đảng Cộng sản Việt Nam giương cao ngọn cờ tư tưởng Hồ Chí Minh, nhưng chỉ nói mà không làm”... Đây là suy nghĩ của những người thiếu thiện chí, cố tình xuyên tạc đường lối đổi mới do Đảng Cộng sản Việt Nam khởi xướng và lãnh đạo, nhưng lại núp dưới danh nghĩa “tự do ngôn luận, dân chủ, xây dựng, góp ý!?.”\nLời nói và hành động của họ bộc lộ thái độ hằn học,

>Load environtment

In [4]:
load_dotenv()

True

In [5]:
help(SemanticChunker)

Help on class SemanticChunker in module langchain_experimental.text_splitter:

class SemanticChunker(langchain_core.documents.transformers.BaseDocumentTransformer)
 |  SemanticChunker(embeddings: langchain_core.embeddings.embeddings.Embeddings, buffer_size: int = 1, add_start_index: bool = False, breakpoint_threshold_type: Literal['percentile', 'standard_deviation', 'interquartile', 'gradient'] = 'percentile', breakpoint_threshold_amount: Optional[float] = None, number_of_chunks: Optional[int] = None, sentence_split_regex: str = '(?<=[.?!])\\s+', min_chunk_size: Optional[int] = None)
 |  
 |  Split the text based on semantic similarity.
 |  
 |  Taken from Greg Kamradt's wonderful notebook:
 |  https://github.com/FullStackRetrieval-com/RetrievalTutorials/blob/main/tutorials/LevelsOfTextSplitting/5_Levels_Of_Text_Splitting.ipynb
 |  
 |  All credits to him.
 |  
 |  At a high level, this splits into sentences, then groups into groups of 3
 |  sentences, and then merges one that are simi

> LOAD MODEL EMBEDDING

In [6]:
MODEL_EMBEDDING=os.getenv("MODEL_EMBEDDING")
embeddings = HuggingFaceEmbeddings(
    model_name=  MODEL_EMBEDDING                                              
)

> Senmatic Chunking

In [7]:
text_splitter = SemanticChunker(
    embeddings=embeddings,  
    buffer_size=4,  
    breakpoint_threshold_type="gradient", 
    breakpoint_threshold_amount=0.75,  
    min_chunk_size=550, 
)

> Execute Sematic Chunking

In [8]:
documents_with_embeddings = []
for doc in documents:
    metadata = {
        "id": doc["id"],
        "keyword": doc["keyword"],
        "keyword_sub": doc["keyword_sub"],
        "word_type": doc["word_type"],
        "link": doc["link"]
    }
    document = Document(page_content=doc["content"], metadata=metadata)
    chunks = text_splitter.split_documents([document])
    documents_with_embeddings.extend(
        Document(page_content=chunk.page_content, metadata=metadata) for chunk in chunks
    )


In [9]:
# Sau khi tạo xong documents_with_embeddings
for i in range(min(3, len(documents_with_embeddings))):
    document = documents_with_embeddings[i]
    print(f"Ví dụ {i + 1}:")
    print(f"Nội dung chunk: {document.page_content}") 
    print(f"Metadata: {document.metadata}")
    print("=" * 40)

Ví dụ 1:
Nội dung chunk: Giá trị lý luận và thực tiễn của tư tưởng Hồ Chí Minh
TTXVN giới thiệu bài viết của Phó giáo sư-tiến sỹ Nguyễn Hoàng Giáp với nhan đề “Giá trị lý luận và thực tiễn của tư tưởng Hồ Chí Minh-Vấn đề không thể phủ nhận.”
Gần đây, trên một số trang thông tin có đăng tải bài viết với nội dung luận bàn về “Những ảo tưởng của Đảng Cộng sản Việt Nam về tư tưởng Hồ Chí Minh”; hoặc: “Đảng Cộng sản Việt Nam giương cao ngọn cờ tư tưởng Hồ Chí Minh, nhưng chỉ nói mà không làm”... Đây là suy nghĩ của những người thiếu thiện chí, cố tình xuyên tạc đường lối đổi mới do Đảng Cộng sản Việt Nam khởi xướng và lãnh đạo, nhưng lại núp dưới danh nghĩa “tự do ngôn luận, dân chủ, xây dựng, góp ý!?.”
Lời nói và hành động của họ bộc lộ thái độ hằn học, động cơ kích động, âm mưu gây mất ổn định chính trị, phá hoại công cuộc xây dựng đất nước của nhân dân ta.
Metadata: {'id': 1, 'keyword': 'Tư tưởng Hồ Chí Minh', 'keyword_sub': 'không có nguyên lý', 'word_type': 'động từ tiêu cực', 'link': 

> Create Collection

In [10]:
qdrant = QdrantVectorStore.from_documents(
    documents=documents_with_embeddings,
    embedding=embeddings,  
    url="http://localhost:6333",
    collection_name="document_embeddings_550_word_4_bkai",
    retrieval_mode=RetrievalMode.DENSE,
    metadata_payload_key="metadata",
    batch_size=512,
    timeout=1000 ,
)


In [12]:
query="Tư tưởng hồ chí minh là gì ?"
result=qdrant.similarity_search_with_score(query,k=5)
for i,(document,score) in enumerate(result):
    print(f"Kết quả {i+1}:")
    print(f"Score: {score}")
    print(f"Content: {document.page_content}")
    print(f"Metadata: {document.metadata}")
    print("=" * 40)

Kết quả 1:
Score: 0.74166125
Content: Định nghĩa về tư tưởng Hồ Chí Minh như trên thể hiện bốn nội dung chủ yếu sau:
- Tư tưởng Hồ Chí Minh là một hệ thống quan điểm toàn diện và sâu sắc về những vấn đề cơ bản của cách mạng Việt Nam, từ cách mạng dân tộc dân chủ nhân dân đến cách mạng xã hội chủ nghĩa. - Tư tưởng Hồ Chí Minh là kết quả của sự vận dụng sáng tạo và phát triển chủ nghĩa Mác- Lênin vào điều kiện cụ thể của nước ta. - Tư tưởng Hồ Chí Minh là sự kết hợp tinh hoa dân tộc và trí tuệ thời đại nhằm giải phóng dân tộc, giải phóng giai cấp và giải phóng con người. - Tư tưởng Hồ Chí Minh là ngọn cờ thắng lợi của cách mạng Việt Nam trong suốt hơn 70 năm qua và tiếp tục soi sáng con đường chúng ta tiến lên  xây dựng một nước Việt Nam hòa bình, thống nhất, độc lập, dân chủ và xã hội chủ nghĩa, dưới sự lãnh đạo của Đảng cộng sản Việt Nam.
Metadata: {'id': 29, 'keyword': 'Tư tưởng Hồ Chí Minh', 'keyword_sub': 'khái quát hóa', 'word_type': 'động từ tích cực', 'link': 'https://thanhniendu